In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import json

import odc.geo
from odc.stac import configure_rio, load
from pystac_client import Client

In [3]:
catalog = "https://cmr.earthdata.nasa.gov/cloudstac/LPCLOUD/"

# Searching across both landsat and sentinel at 30 m
collections = ["HLSS30.v2.0", "HLSL30.v2.0"]

client = Client.open(catalog)

In [4]:
# BBOX over Precipitous Bluff in Tasmania
ll = (-43.55, 146.45)
ur = (-43.35, 146.75)
bbox = [ll[1], ll[0], ur[1], ur[0]]

# Search for items in the collection
items = client.search(collections=collections, bbox=bbox, datetime="2023-07-01/2023-09-30").items()

In [5]:
items = [i for i in items]
print(f"Found {len(items)} items")

Found 109 items


In [6]:


with open("secrets.json") as f:
    data = json.load(f)
    token = data["earthdata"]["token"]

In [7]:
from utils import hls_config

In [8]:
# Configure GDAL. You need to export your earthdata token as an environment variable.
header_string = f"Authorization: Bearer {token}"
configure_rio(cloud_defaults=True, GDAL_HTTP_HEADERS=header_string)

data = load(
    items,
    bbox=bbox,
    crs="epsg:6933",
    resolution=30,
    chunks={"x": 2500, "y": 2500, "time": 1},
    groupby="solar_day",
    stac_cfg=hls_config,
    bands=["red", "green", "blue", "nir", "fmask"]
)

In [ ]:
# flags_definition = {
#     {"cloud": {"bits": 1, "values": {0: "no", 1: "yes"}}},
#     {"cloud_or_shadow_adjacent": {"bits": 2, "values": {0: "no", 1: "yes"}}},
#     {"cloud_shadow": {"bits": 3, "values": {0: "no", 1: "yes"}}},
#     {"snow_ice": {"bits": 4, "values": {0: "no", 1: "yes"}}}
#     {"water": {"bits": 5, "values": {0: "no", 1: "yes"}}}
# }

# Want to mask cloud and cloud shadow
mask_str = "00001010"
mask_int = int(mask_str, base=2)

In [ ]:
# Show the raw data
data[["red", "green", "blue"]].isel(time=slice(0, 12)).to_array().plot.imshow(
    col="time", col_wrap=4, vmin=0, vmax=3000
)

In [ ]:
# Show it masked
masked[["red", "green", "blue"]].isel(time=slice(0, 12)).to_array().plot.imshow(
    col="time", col_wrap=4, vmin=0, vmax=3000
)

In [ ]:
# Create a simple cloud-free median now we have masked data
median = masked.median("time").compute()

In [ ]:
# Plot the median. This is just one month, so we expect
# some areas to be missing due to clouds
rgb = median[["red", "green", "blue"]].to_array()
rgb.plot.imshow(size=10, vmin=0, vmax=1000)

In [ ]:
rgba = median.odc.to_rgba(bands=["red", "green", "blue"], vmin=0, vmax=1000)

rgba.odc.write_cog("median_rgba.tif", overwrite=True)